# Extracting data from OANDA APIs

In this notebook we're going to learn how to extract data from the OANDA API and then moving on to do some exploratory data analysis using the powerful Pandas library. The instrument we use is EUR_USD and is based on the EUR/USD exchange rate.

In [1]:
import pandas as pd

## Steps to get the data from the API:

1. You'll need an API accesstoken along with your account_id to get authorized for API consumption.
2. Pass the parameters which include the duration and granularity for which the data is to be requested.
3. There are specific codes for each instrument that you want to analyse which are mentioned in OANDA docs.
4. Request for all the candles data and you'll get a JSON data as the response.

In [ ]:
from oandapyV20 import API
import datetime
import oandapyV20.endpoints.instruments as instruments

# put in your API credentials here:
api = API(access_token = "28da5c58c1b4237ca07022b992221924-76ffcf7b53ae61ca812b17b09bbc7d20")
account_id = "101-001-9842268-001"

# define the parameters which include the dates for which the 
# data is to be requested along with the granularity
params = {
    "from": "2018-11-01",
    "to": "2018-11-04",
    "granularity": "M1"
}

r = instruments.InstrumentsCandles(instrument = 'EUR_USD', params=params)
api.request(r)
data = r.response

In [ ]:
# let's see what have we got in the response
print(data)

Make a pandas dataframe of the candles data using pandas which will make it easier to maneuver the data and implement our trading strategy.

In [ ]:
# looking at the data, let's create the dataframe of all the candles
df = pd.DataFrame(data['candles'])
df.head()
# drop the mid column from the original dataframe after 
# converting the json into series object and concatenating in the original dataframe

# data_df = pd.concat([df.drop(['mid'], axis=1),
#            df.mid.apply(pd.Series)], axis=1)
# data_df.head()

type(df['mid'][1])

# ============================================================

# Extracting data from Quandl API

In [2]:
# requesting the end of day stock prices of Apple stocks from quandle API.
import quandl as q
q.ApiConfig.api_key = "yuqp72Y_-GpAsrjQEXfL"
apple_data = q.get("EOD/AAPL", start_date="2010-01-01", end_date="2018-11-30")
apple_data.to_csv('../data/apple_stock_eod_prices.csv', index=True)

In [3]:
msft_df = q.get("EOD/MSFT", start_date="2010-01-01", end_date="2018-11-30")
msft_df.to_csv('../data/msft_eod.csv', index=True)

## Convert the time column into a datetime object column and set it as the index

In [ ]:
df['time'] = df['time'].astype('datetime64[s]')
df = df.set_index('time')
df.info()

## Flatten the mid column json into three different columns named as open(for o), high(for h), low(for l) and close(for c)

In [ ]:
# drop the mid column from the original dataframe after 
# converting the json into series object and concatenating in the original dataframe

data_df = pd.concat([df.drop(['mid'], axis=1),
           df.mid.apply(pd.Series)], axis=1)
data_df.head()

In [ ]:
# renaming the 'o','h','l' and 'c' columns
data_df.rename(columns={'o': 'open',
                        'h': 'high',
                        'l': 'low',
                        'c': 'close'}, inplace = True)
